In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
df = pd.read_csv("/Users/warren/Desktop/docs/classes/stats170/reddit_scraper/notebooks/labled_data_without_removed.csv")
df.tail()

,title,selftext,warren_label(s),justin_labels,melissa_labels,kevin_labels,ally_labels,all_awardings,allow_live_comments,archived,...,crosspost_parent_list,edited_on,media_metadata,author_cakeday,gallery_data,is_gallery,poll_data,call_to_action,link_flair_template_id,date
272,apply to switch majors,i’m trying to switch majors rn into psci from ...,"academic , admissions",admissions,academic,academic,academic,[],False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-21
273,ICS!! Getting ahead on 45C,I am taking a lot of units in the fall and wan...,academic,academic,academic,academic,academic,[],False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-14
274,PubHlth 101 / In4MATx 131,Does anyone have information on these courses?...,academic,academic,academic,academic,academic,[],False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-04
275,headphones reccomendations?,my beats are falling apart ): i was wondering ...,financial,unclassified,social life,unclassified,social life,[],False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-15
276,Tech Fee,Does anyone know how to pay the technology fee...,financial,financial,financial,financial,financial,[],False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-19


In [ ]:
labels = {'unclassified' : 0, 
          'social life': 1,
          'academic': 2,
          'sports': 3,
          'jobs': 4,
          'health': 5,
          'admissions': 6,
          'housing' : 7,
          'financial' : 8,
          'missing items' : 9,
          'transportation' : 10,
          'food' : 11
         }

In [ ]:
cols = ['warren_label(s)', 'justin_labels',
       'melissa_labels', 'kevin_labels', 'ally_labels']

In [ ]:
def get_labels(vals: str, labels=labels):
    vals = vals.split(',')
    nums = []
    for val in vals:
        nums.append(labels[val.strip()])
    
    return nums

In [ ]:
num_labels = [[] for x in range(len(df))]
for col in cols:
    labs = df[col].apply(get_labels)
    for i, lab_list in enumerate(labs):
        num_labels[i].extend(lab_list)
def most_frequent(List):
    return max(set(List), key = List.count)
final_labels = pd.Series(num_labels).apply(most_frequent)

In [ ]:
df['final_labels'] = final_labels

In [ ]:
data = df[['title', 'selftext', 'final_labels']].copy()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
data['text'] = data['title'] + ' ' + data['selftext']

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
data['tokenized'] = data['text'].apply(preprocess_function)

In [ ]:
data[['input_ids','attention_mask']] = pd.DataFrame(data.tokenized.tolist(), index= data.index)

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
data['academic_label'] = data['final_labels'].apply(lambda x: 0 if x != 2 else 1)

In [ ]:
id2label = {0: "Not Academic", 1: "Academic"}
label2id = {"Not Academic": 0, "Academic": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [ ]:
data = data[['text', 'academic_label', 'input_ids', 'attention_mask']].copy()
data.columns = ['text', 'label', 'input_ids', 'attention_mask']
data.head()

,text,label,input_ids,attention_mask
0,NaN,0,"[1, 1, 1]","[101, 16660, 102]"
1,"Will pay to pet dog!!! Hi, this is my first ev...",0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2097, 3477, 2000, 9004, 3899, 999, 999, ..."
2,Majors by #Units Does anyone happen to know of...,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 15279, 2011, 1001, 3197, 2515, 3087, 414..."
3,"friends ? hey everyone, this might be a little...",0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2814, 1029, 4931, 3071, 1010, 2023, 2453..."
4,Beall Innovation Center What is at the Applied...,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 26892, 3363, 8144, 2415, 2054, 2003, 201..."


In [ ]:
test = data.sample(90, random_state=42)
train = data[~data.text.isin(test.text)]

In [ ]:
from datasets import Dataset
test = Dataset.from_pandas(test).remove_columns(["__index_level_0__"])
train = Dataset.from_pandas(train).remove_columns(["__index_level_0__"])

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test,
    eval_dataset=train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/warren/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 90
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Tot

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.671045,0.616883
2,No log,0.678835,0.616883


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 154
  Batch size = 16
Saving model checkpoint to my_awesome_model/checkpoint-6
Configuration saved in my_awesome_model/checkpoint-6/config.json
Model weights saved in my_awesome_model/checkpoint-6/pytorch_model.bin
tokenizer config file saved in my_awesome_model/checkpoint-6/tokenizer_config.json
Special tokens file saved in my_awesome_model/checkpoint-6/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****


TrainOutput(global_step=12, training_loss=0.6919092337290446, metrics={'train_runtime': 216.6231, 'train_samples_per_second': 0.831, 'train_steps_per_second': 0.055, 'total_flos': 13288653378456.0, 'train_loss': 0.6919092337290446, 'epoch': 2.0})